In [ ]:
// using nightly-build
//#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22356.1"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:SandDance.InteractiveExtension, 1.0.53"
//#r "nuget: XPlot.Plotly.Interactive, 4.0.6"

# Exploring your own data - NFL Draft history from 2015 to 2021

OK, Let's do some using statements so that we have tools to work with our data.

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;
using System.IO;

Do our files exist? They were downloaded from [pro-football-reference.com](https://www.pro-football-reference.com/), such as this page for the [2015 Draft](https://www.pro-football-reference.com/years/2015/draft.htm).

In [ ]:
var fileExists = File.Exists("drafts/2015Draft.csv");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

OK.  Make separate data frame objects for each draft from 2015 to 2021. Also, show the 2015 draft, to make sure the data makes sense.

In [ ]:
var draft2015 = DataFrame.LoadCsv("drafts/2015Draft.csv");
var draft2016 = DataFrame.LoadCsv("drafts/2016Draft.csv");
var draft2017 = DataFrame.LoadCsv("drafts/2017Draft.csv");
var draft2018 = DataFrame.LoadCsv("drafts/2018Draft.csv");
var draft2019 = DataFrame.LoadCsv("drafts/2019Draft.csv");
var draft2020 = DataFrame.LoadCsv("drafts/2020Draft.csv");
var draft2021 = DataFrame.LoadCsv("drafts/2021Draft.csv");
draft2021.Sample(15)

Hmmmmm...... the dataFrame has null values (and a blank column named Column29). Let's update some of this information.

- For "To", let's assume the year they were drafted was their last year. That means we need a DraftYear column.
- For null age, let's guess the median age of all the other draftees, rounded down.
- "0" values for all other columns seem fine.

First, Add the DraftYear and make it the year of that draft.

In [ ]:
int currentYear = 2022;
draft2021.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2021.Rows.Count()));
draft2021.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2021.Rows.Count()));
draft2021["DraftYear"].FillNulls(2021, true);
draft2021["To"].FillNulls(2021, true);
draft2021["YearsSinceDraft"].FillNulls(currentYear - 2021, true);

draft2020.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2020.Rows.Count()));
draft2020.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2020.Rows.Count()));
draft2020["DraftYear"].FillNulls(2020, true);
draft2020["To"].FillNulls(2020, true);
draft2020["YearsSinceDraft"].FillNulls(currentYear - 2020, true);

draft2019.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2019.Rows.Count()));
draft2019.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2019.Rows.Count()));
draft2019["DraftYear"].FillNulls(2019, true);
draft2019["To"].FillNulls(2019, true);
draft2019["YearsSinceDraft"].FillNulls(currentYear - 2019, true);

draft2018.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2018.Rows.Count()));
draft2018.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2018.Rows.Count()));
draft2018["DraftYear"].FillNulls(2018, true);
draft2018["To"].FillNulls(2018, true);
draft2018["YearsSinceDraft"].FillNulls(currentYear - 2018, true);

draft2017.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2017.Rows.Count()));
draft2017.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2017.Rows.Count()));
draft2017["DraftYear"].FillNulls(2017, true);
draft2017["To"].FillNulls(2017, true);
draft2017["YearsSinceDraft"].FillNulls(currentYear - 2017, true);

draft2016.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2016.Rows.Count()));
draft2016.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2016.Rows.Count()));
draft2016["DraftYear"].FillNulls(2016, true);
draft2016["To"].FillNulls(2016, true);
draft2016["YearsSinceDraft"].FillNulls(currentYear - 2016, true);

draft2015.Columns.Add(new PrimitiveDataFrameColumn<int>("DraftYear", draft2015.Rows.Count()));
draft2015.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsSinceDraft", draft2015.Rows.Count()));
draft2015["DraftYear"].FillNulls(2015, true);
draft2015["To"].FillNulls(2015, true);
draft2015["YearsSinceDraft"].FillNulls(currentYear - 2015, true);

draft2021

OK, let's get the median age of a drafted player (from 2021), then insert that age whenever we don't have that data.

In [ ]:
var medianAge = draft2021["Age"].Median();
int age = (int)medianAge;
draft2021["Age"].FillNulls(age, true);
draft2020["Age"].FillNulls(age, true);
draft2019["Age"].FillNulls(age, true);
draft2018["Age"].FillNulls(age, true);
draft2017["Age"].FillNulls(age, true);
draft2016["Age"].FillNulls(age, true);
draft2015["Age"].FillNulls(age, true);

Now that the ages are filled out, let's put all these drafts together into one big DataFrame.

In [ ]:
var drafts2015To2021 = draft2015.Clone();
DataFrame[] remainingDrafts = new DataFrame[] {draft2016, draft2017, draft2018, draft2019, draft2020, draft2021};
foreach (var draft in remainingDrafts)
{
    draft.Rows.ToList<DataFrameRow>()
        .ForEach(row => 
            {
                drafts2015To2021.Append(row, /*append in place*/ true);
            }
            );
}


// Remove Column29.
drafts2015To2021.Columns.Remove("Column29");

Console.Write("Number of rows in combined dataframe: ");
drafts2015To2021.Rows.Count()

All other null values can be zero.

Once we have all the data filled out, let's also list the career length for players who are already done playing (They have a "To" year before 2021).

For good measure, let's show the average number of games they player per year of their career.

In [ ]:
drafts2015To2021["Pos"].FillNulls("XX", true);

// Possible null rows are: wAV,DrAV,G,PassCmp,PassAtt,PassYds,PassTD,PassInt,RushAtt,RushYds,RushTD,Rec,RecYds,RecTD,SoloTak,Int,Sk
drafts2015To2021["wAV"].FillNulls(0, true);
drafts2015To2021["DrAV"].FillNulls(0, true);
drafts2015To2021["G"].FillNulls(0, true);
drafts2015To2021["PassCmp"].FillNulls(0, true);
drafts2015To2021["PassAtt"].FillNulls(0, true);
drafts2015To2021["PassYds"].FillNulls(0, true);
drafts2015To2021["PassTD"].FillNulls(0, true);
drafts2015To2021["PassInt"].FillNulls(0, true);
drafts2015To2021["RushAtt"].FillNulls(0, true);
drafts2015To2021["RushYds"].FillNulls(0, true);
drafts2015To2021["RushTD"].FillNulls(0, true);
drafts2015To2021["Rec"].FillNulls(0, true);
drafts2015To2021["RecYds"].FillNulls(0, true);
drafts2015To2021["RecTD"].FillNulls(0, true);
drafts2015To2021["SoloTak"].FillNulls(0, true);
drafts2015To2021["Int"].FillNulls(0, true);
drafts2015To2021["Sk"].FillNulls("0", true);

// Create One More column showing years in career
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<int>("YearsInCareer", drafts2015To2021.Rows.Count()));
drafts2015To2021["YearsInCareer"] = (drafts2015To2021["To"] - drafts2015To2021["DraftYear"]) + 1;

// Average Games played per year
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<double>("GamesPerYearAvg", drafts2015To2021.Rows.Count()));
drafts2015To2021["GamesPerYearAvg"] = (drafts2015To2021["G"] / drafts2015To2021["YearsInCareer"]);

// Average Value per year of career
drafts2015To2021.Columns.Add(new PrimitiveDataFrameColumn<double>("ValuePerYear", drafts2015To2021.Rows.Count()));
drafts2015To2021["ValuePerYear"] = (drafts2015To2021["wAV"] / drafts2015To2021["YearsInCareer"]);



DataFrame.WriteCsv(drafts2015To2021,"drafts2015To2021.csv",',');

drafts2015To2021
//draft2021.ToTabularDataResource().ExploreWithSandDance().Display()